In [1]:
# Importacao das libraries
import pandas as pd
import functions as main
import numpy as np
from scipy import stats
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [46]:
ciclo = 1

opencsv = pd.read_csv(rf"D:\OneDrive - Instituto Politecnico de Castelo Branco\002_Learning\001_Pos_Graduacao\Ciclo 5 - Projeto Final\ProjetoFinal-PosGraduacaoEngenhariaFinanceira\ciclo{ciclo}_df.csv", sep=",")
opencsv.set_index('Date', inplace=True)
opencsv.index = pd.to_datetime(opencsv.index)
opencsv = opencsv.asfreq(pd.infer_freq(opencsv.index))
opencsv = opencsv.astype(float)

In [47]:
df_model = opencsv.copy()
df_model.drop(columns=["BTC_USD"])

,BalanceSheet_global,Mining_Difficulty,USD_OnChain_Volume,GDP_usa,HashRate,M2_Liquidity,Dollar_Index,QQQ,GLD,TLT,...,L1.Miners_Revenue_USD,L2.Miners_Revenue_USD,L1.Nr_Tx_Block,L2.Nr_Tx_Block,L1.Nr_Transactions,L2.Nr_Transactions,L1.Nr_Addresses,L2.Nr_Addresses,L1.BTC_Supply,L2.BTC_Supply
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-28,4.073849,4.342667,4.337569,4.187604,-0.748699,3.947840,1.892818,1.692658,2.114878,1.808035,...,3.584673,3.584673,0.835169,0.843381,3.066699,3.101059,3.181844,3.147058,6.720507,6.720507
2011-02-04,4.073849,4.342667,4.365424,4.188088,-0.682133,3.947840,1.892317,1.705330,2.119454,1.797925,...,3.584673,3.584673,0.835169,0.835169,3.066699,3.066699,3.147058,3.181844,6.720507,6.720507
2011-02-11,4.074919,4.414938,4.738280,4.188572,-0.607583,3.947840,1.894648,1.713280,2.121626,1.797925,...,3.887735,3.584673,0.835169,0.835169,3.066699,3.066699,3.147058,3.147058,6.725287,6.720507
2011-02-18,4.073983,4.561815,4.653978,4.189055,-0.407285,3.948706,1.890197,1.715429,2.131651,1.797925,...,4.001930,3.887735,1.102478,0.835169,3.371991,3.066699,3.565848,3.147058,6.730532,6.725287
2011-02-25,4.076774,4.561815,4.453966,4.189537,-0.395386,3.948706,1.888067,1.707369,2.137924,1.811258,...,4.050595,4.001930,0.986687,1.102478,3.313023,3.371991,3.475962,3.565848,6.736508,6.730532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,4.107322,6.487467,6.795815,4.216772,1.390739,4.009013,1.903470,1.768834,2.219925,1.959148,...,4.997749,5.057523,2.272500,2.284590,4.399605,4.457776,4.463325,4.547036,7.008719,7.006517
2012-11-02,4.111034,6.519087,6.557299,4.217234,1.376901,4.009459,1.906281,1.767636,2.211121,1.957800,...,5.274498,4.997749,2.123495,2.272500,4.340979,4.399605,4.444451,4.463325,7.011006,7.008719
2012-11-09,4.111992,6.519087,6.446799,4.217695,1.376901,4.015921,1.908646,1.755883,2.224844,1.970817,...,5.050997,5.274498,2.306912,2.123495,4.462248,4.340979,4.420517,4.444451,7.013257,7.011006


In [ ]:
df_model.isnull().sum()

## Matriz de Correlacoes


In [ ]:
test_corr = df_model.copy()
test_corr = test_corr.iloc[:, :18] # apenas as variaveis originais
test_corr.shape

In [ ]:
main.correlation_map(df=test_corr, threshold=0.3)

## Seleção de Variáveis

Considerar técnicas como regressão lasso, ridge ou elastic net, que fazem seleção de variáveis automaticamente, pode ser útil se você suspeitar que nem todas as 20 variáveis são relevantes.

In [4]:
df_model.columns

Index(['BalanceSheet_global', 'BTC_USD', 'Mining_Difficulty',
       'USD_OnChain_Volume', 'GDP_usa', 'HashRate', 'M2_Liquidity',
       'Dollar_Index', 'QQQ', 'GLD', 'TLT', '10YR_USA', 'VIX', 'OIL_USD',
       'Miners_Revenue_USD', 'Nr_Tx_Block', 'Nr_Transactions', 'Nr_Addresses',
       'BTC_PageViews', 'Crypto_PageViews', 'BTC_Supply',
       'L1.BalanceSheet_global', 'L2.BalanceSheet_global', 'L1.BTC_USD',
       'L2.BTC_USD', 'L1.Mining_Difficulty', 'L2.Mining_Difficulty',
       'L1.USD_OnChain_Volume', 'L2.USD_OnChain_Volume', 'L1.GDP_usa',
       'L2.GDP_usa', 'L1.HashRate', 'L2.HashRate', 'L1.M2_Liquidity',
       'L2.M2_Liquidity', 'L1.Dollar_Index', 'L2.Dollar_Index', 'L1.QQQ',
       'L2.QQQ', 'L1.GLD', 'L2.GLD', 'L1.TLT', 'L2.TLT', 'L1.10YR_USA',
       'L2.10YR_USA', 'L1.VIX', 'L2.VIX', 'L1.OIL_USD', 'L2.OIL_USD',
       'L1.Miners_Revenue_USD', 'L2.Miners_Revenue_USD', 'L1.Nr_Tx_Block',
       'L2.Nr_Tx_Block', 'L1.Nr_Transactions', 'L2.Nr_Transactions',
       'L1.Nr_Ad

<div class="alert alert-block alert-info"><b>Hipoteses a testar</b>

H1: Ceteris paribus, Bitcoin é um ativo de especulação.

H2: Ceteris paribus, a performance de Bitcoin é influenciada por fatores macroeconómicos.

H3: Ceteris paribus, quanto mais liquidez há no mercado, maior é a apreciação de preço de Bitcoin.

H4: Ceteris paribus, os fatores endógenos de Bitcoin são os maiores explicadores da performance de Bitcoin.
</div>

## Regressao Linear Multipla

In [48]:
# Yt = b0 + b1X1t + b2X2t + εt

#   L1.
#   L2.


# Modelos endogenos Bitcoin
#   'Mining_Difficulty', 'USD_OnChain_Volume', 'HashRate', 'Miners_Revenue_USD', 
#   'Nr_Tx_Block', 'Nr_Transactions', 'Nr_Addresses', 'BTC_Supply'
model1a = ['Mining_Difficulty', 'USD_OnChain_Volume', 'BTC_Supply', 'Nr_Transactions']
model1b = ['Nr_Transactions', 'L1.USD_OnChain_Volume', 'L1.BTC_Supply', 'L1.Mining_Difficulty']
model1c = ['Nr_Tx_Block', 'L2.USD_OnChain_Volume']


# Modelos Macro Economicos
#   'GDP_usa', 'Dollar_Index', 'QQQ', 'GLD', 'TLT', '10YR_USA', 'VIX'
#   'BalanceSheet_global', 'M2_Liquidity', 'OIL_USD'
model2a = [ 'QQQ', 'OIL_USD', 'GLD', 'BalanceSheet_global']
model2b = [ 'GLD', 'VIX', 'M2_Liquidity', 'TLT']
model2c = [ 'TLT', 'GDP_usa', 'BalanceSheet_global', 'L1.GLD']
model2d = [ '10YR_USA', 'L1.VIX', 'L1.OIL_USD', 'Dollar_Index' ]


# Modelos Gerais
#   'BTC_PageViews', 'Crypto_PageViews', 'L1.BTC_USD'
model3a = [  'VIX', '10YR_USA', 'HashRate', 'BTC_Supply'] #'BTC_PageViews',
model3b = [  'Nr_Transactions', 'L1.QQQ', 'M2_Liquidity'] # 'L1.BTC_PageViews', 'Crypto_PageViews',
model3c = [  'L2.GLD', 'BalanceSheet_global', 'OIL_USD', 'Miners_Revenue_USD', 'USD_OnChain_Volume'] #'L2.BTC_PageViews',


X = df_model[model1b]
y = opencsv["BTC_USD"]

X = sm.add_constant(X) # Add a constant term to the predictors

model = sm.OLS(y, X)
results = model.fit() # Fit the OLS model

#results.summary() # Print the summary of the regression results

### Auto run Regressions

In [49]:
models_list = [model1a, model1b, model1c, model2a, model2b, 
               model2c, model2d, model3a, model3b, model3c]

models_columns = ['model_1a', 'model_1b', 'model_1c', 'model_2a', 'model_2b', 
               'model_2c', 'model_2d', 'model_3a', 'model_3b', 'model_3c']

def run_models_stats(df, models_list, y):
        count = 0
        for model_columns in models_list:
            X = df[model_columns]

            X = sm.add_constant(X)  # Add a constant term to the predictors

            model = sm.OLS(y, X)
            results = model.fit()  # Fit the OLS model
            
            print("\n##################################################")
            print("\nModel:", models_columns[count]) # columns name
            print("\nR-Squared:", results.rsquared_adj)
            #print("Variable:", results.params.index) # set as index
            #print("\nCoeff:", results.params) # coefficients
            #print("\nP-Value:", results.pvalues) # p-values
            print("No. Obs.:", results.nobs)
            print("Cond. no.:", results.condition_number)
            count += 1
            print("\n##################################################")


def run_models(df, models_list, y):
    f = open(f"OLS_Results_Ciclo{ciclo}.txt", "w")
    count = 0
    for model_columns in models_list:
        X = df[model_columns]

        X = sm.add_constant(X)  # Add a constant term to the predictors

        model = sm.OLS(y, X)
        results = model.fit()  # Fit the OLS model
        
        f.write("\n####################################################################################################### \n\n")
        f.write(models_columns[count])
        f.write(" \n")
        f.write(f"{results.summary()}")
        count += 1
        f.write("\n####################################################################################################### \n\n")
    
    f.close()

In [50]:
y = opencsv["BTC_USD"]
run_models(df_model, models_list, y)

In [ ]:
#run_models_stats(df_model, models_list, y)

### Exemplo Data Frame

In [ ]:
data_store = pd.DataFrame()
data_store.index = df_model.columns


models_columns = ['model1a', 'model1b', 'model1c', 'model1d', 'model2a', 'model2b', 
               'model2c', 'model2d', 'model2e', 'model3a', 'model3b', 'model3c']

# Add new columns with a default value (e.g., None or 0)
for model in models_columns:
    data_store[model] = None  # or some other default value

data_store

In [ ]:
# list of all params 
dir(results)

https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html

# Testes

### Homocesdasticity

In [ ]:
import statsmodels.formula.api as smf

# https://www.geeksforgeeks.org/how-to-perform-a-breusch-pagan-test-in-python/

### Normality of Residuals

Assumption: The residuals should be approximately normally distributed.

Test:
Q-Q Plot: Compare the quantiles of residuals to a normal distribution. If residuals are normal, the points will lie approximately on the 45-degree line.

Shapiro-Wilk Test or Kolmogorov-Smirnov Test: Statistical tests for normality of residuals.

Transformation: If residuals are not normally distributed, consider transforming the dependent variable or using robust regression methods.

### Independence of Errors


Assumption: The residuals (errors) should be independent of each other.

Test:

Durbin-Watson Test: Detects the presence of autocorrelation in residuals, especially in time series data.

Plot Residuals vs. Time: For time series data, plot residuals against time to detect patterns or autocorrelation.

Transformation:
Add Lagged Variables: For time series, include lagged values of the dependent variable as predictors.

### No Omitted Variable Bias

Assumption: All relevant independent variables are included in the model.

Test:

Ramsey RESET Test: Checks for omitted variables by testing for non-linearity in the model.

Remedies:

Include Relevant Variables: Ensure all theoretically important variables are included in the model.
Use Model Selection Criteria: Such as AIC or BIC to guide variable selection.

## Teste de Multicolinearidade


Como você tem muitas variáveis, é importante verificar a multicolinearidade, especialmente porque as variáveis acumuladas podem estar altamente correlacionadas entre si. Testes como o VIF (Variance Inflation Factor) podem ajudar a identificar problemas.

Multicollinearity describes the state where the independent variables used in a study exhibit a strong relationship with each other. This can pose a problem in many cases as you would normally want your independent variables to be… independent.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = 

# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

vif_data

#       Interpreting the VIF Values:
#       VIF < 5: Generally considered acceptable, indicating low to moderate multicollinearity.
#       VIF between 5 and 10: Indicates moderate to high multicollinearity, which may be problematic, but not necessarily requiring immediate removal.
#       VIF > 10: Strong multicollinearity, typically suggesting that the variable should be considered for removal or that further investigation is needed.